<a href="https://colab.research.google.com/github/vksepm/hands-on-agentic-ai-building-ai-agents-with-llamaindex-3962125/blob/main/code_02_XX%20A%20Healthcare%20assistant%20agent%20with%20ReAct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU python-dotenv
!pip install -qU llama-index
!pip install -qU llama-index-llms-openai
!pip install -qU llama-index-embeddings-openai
!pip install -qU llama-index-llms-azure-openai
!pip install -qU llama-index-embeddings-azure-openai
!pip install -qU llama-index-agent-introspective
!pip install -qU llama-index-program-openai
!pip install -qU llama-index-readers-file
!pip install -qU pydantic
!pip install -qU llama-index-core
!pip install -qU llama-index-utils-workflow
!pip install -qU llama-index-tools-wikipedia
!pip install -qU llama-index-readers-json
!pip install -qU llama-index-readers-file
!pip install -qU pymupdf
!pip install -qU llama-index-tools-google
!pip install -qU nbformat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.3/251.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━

In [5]:
%%bash
git clone https://github.com/vksepm/hands-on-agentic-ai-building-ai-agents-with-llamaindex-3962125.git
mv ./hands-on-agentic-ai-building-ai-agents-with-llamaindex-3962125/* .
rm -rf ./hands-on-agentic-ai-building-ai-agents-with-llamaindex-3962125/

Cloning into 'hands-on-agentic-ai-building-ai-agents-with-llamaindex-3962125'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 92 (delta 34), reused 44 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (92/92), 479.59 KiB | 3.18 MiB/s, done.
Resolving deltas: 100% (34/34), done.


### Set up LLM and Embedding

In [11]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import Settings
import os
import nest_asyncio
from google.colab import userdata



#Used by LlamaIndex
nest_asyncio.apply()

#Setup the environment variables
os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get('AZURE_OPENAI_ENDPOINT')
os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')
os.environ["OPENAI_API_VERSION"] = userdata.get('OPENAI_API_VERSION')

#Note : Function calling support only available in GPT-4+
#Create the LLM object
Settings.llm=AzureOpenAI(
    model="gpt-4o-mini",
    deployment_name="gpt-4o-mini",
    api_version="2024-08-01-preview",
)

#Set up the embedding model for RAG
Settings.embed_model = AzureOpenAIEmbedding(
    model="text-embedding-3-small",
    api_key=userdata.get('AZURE_OPENAI_API_KEY'),
    azure_endpoint=userdata.get('AZURE_OPENAI_ENDPOINT'),
    api_version="2023-05-15",
)

### 02.03. Set up wikipedia tools

In [4]:
from llama_index.tools.wikipedia import WikipediaToolSpec
from llama_index.core.tools.tool_spec.load_and_search import (
    LoadAndSearchToolSpec,
)

# Get list of wikipedia tools
wiki_spec = WikipediaToolSpec()
tool = wiki_spec.to_tool_list()[1]
wiki_tools = LoadAndSearchToolSpec.from_defaults(tool).to_tool_list()

#Review the list of tools
for tool in wiki_tools:
    print( "--------\n",tool.metadata.name, tool.metadata.description)

--------
 search_data 
        Use this tool to load data from the following function. It must then be read from
        the corresponding read_search_data function.

        search_data(query: str, lang: str = 'en') -> str

        Search Wikipedia for a page related to the given query.
        Use this tool when `load_data` returns no results.

        Args:
            query (str): the string to search for
        
    
--------
 read_search_data 
        Once data has been loaded from search_data it can then be read using a natural
        language query from this function.

        You are required to pass the natural language query argument when calling this endpoint

        Args:
            query (str): The natural language query used to retreieve information from the index
    


### 02.04. Set up RAG for medications

In [12]:
#from llama_index.core import SimpleDirectoryReader
#from llama_index.readers.file import CSVReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import  VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.readers.json import JSONReader

medication_docs = JSONReader().load_data(
        input_file="datasets/Medications and dosages.json"
)
#Split the document
splitter=SentenceSplitter(chunk_size=200)
medication_nodes=splitter.get_nodes_from_documents(medication_docs)

#Index the document in memory
medication_index=VectorStoreIndex(medication_nodes)

#Create a query engine tool based on the CSV file
medication_query_engine = medication_index.as_query_engine()
medication_tool = QueryEngineTool.from_defaults(
    query_engine=medication_query_engine,
    description=(
        "Provides list of popular medications, diseases they are used to treat for and recommended dosages"
    ),
)

### 02.05. Set up & execute the Healthcare Agent

In [ ]:
from llama_index.core.agent.react.base import ReActAgent
#Consolidate list of tools
healthcare_tools =wiki_tools + [medication_tool]

context = """
You are an healthcare assistant who can answer questions regarding
diseases, their symptoms and medications for them.

Use the wikipedia tools to answer questions about symptoms and possible
diseases associated with those symptoms.
Use the medication tool to find list of medications and dosages for a given
disease.

Use only the tools provided to answer questions and NOT your own memory.

"""

#Create a pre-built ReAct agent
healthcare_agent = ReActAgent.from_tools(
                healthcare_tools,
                llm=Settings.llm,
                context=context,
                verbose=True #Set verbose for detailed logs
                )


In [ ]:
response=healthcare_agent.chat("Which medication should i take for arthritis?")
print("*******\n Response : ",response.response)

> Running step 291b4a1a-8c55-451f-b207-65a2adb4400b. Step input: Which medication should i take for arthritis?
Thought: The current language of the user is English. I need to use a tool to find the medications for arthritis.
Action: query_engine_tool
Action Input: {'input': 'medications for arthritis'}
Observation: Naproxen and Methotrexate are medications used for arthritis.
> Running step afed2f55-bdd6-4477-9c2f-e7bcbebe9941. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: For arthritis, Naproxen and Methotrexate are commonly used medications.
*******
 Response :  For arthritis, Naproxen and Methotrexate are commonly used medications.
